In [1]:
import os
import qdrant_client

from charles_dicken_qa_chatbot.constants import *

from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.fastembed import FastEmbedEmbedding
from llama_index.llms.openai import OpenAI

import nest_asyncio
from dotenv import load_dotenv


nest_asyncio.apply()

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

/Users/hmnguyen1067/Downloads/Github/Charles-Dicken-QA-chatbot/.pixi/envs/default/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
client = qdrant_client.QdrantClient(host=QDRANT_HOST, port=QDRANT_PORT)
aclient = qdrant_client.AsyncQdrantClient(host=QDRANT_HOST, port=QDRANT_PORT)

vector_store = QdrantVectorStore(
    client=client,
    aclient=aclient,
    enable_hybrid=True,
    fastembed_sparse_model="Qdrant/bm42-all-minilm-l6-v2-attentions",
    collection_name=COLLECTION_NAME,
)

In [3]:
embed_model = FastEmbedEmbedding(model_name=EMBED_MODEL)
llm = OpenAI(model=LLM_MODEL)

# Retrieval

In [ ]:
index = VectorStoreIndex.from_vector_store(
    vector_store, use_async=True, embed_model=embed_model
)

query_engine = index.as_query_engine(
    similarity_top_k=2,
    sparse_top_k=8,
    vector_store_query_mode="hybrid",
    hybrid_top_k=3,
    llm=llm,
)

query_str = "What is 'A Christmas Carol' novel's main theme?"

## HyDE

In [5]:
from llama_index.core.indices.query.query_transform.base import (
    HyDEQueryTransform,
)
from llama_index.core.query_engine import TransformQueryEngine

In [ ]:
hyde = HyDEQueryTransform(llm, include_original=True)
hyde_query_engine = TransformQueryEngine(query_engine, query_transform=hyde)

In [13]:
query_str = "What is 'A Christmas Carol' novel's main theme?"

response = hyde_query_engine.query(query_str)
print(response)

The central theme is Scrooge’s transformation and redemption—his change from selfishness and indifference to generosity, compassion, and social responsibility. The novella frames this both as a Christian-style allegory of repentance and as a secular call for charity and concern for the poor (especially children), embodied by figures like Tiny Tim and the allegories of Want and Ignorance.
